In [1]:
import os
os.chdir("../")
%pwd

'e:\\Turja\\MLOps\\Chest-Cancer-Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_learning_rate: float
    params_weights: str
    params_classes: int
    params_device: str

In [3]:
from ChestCancerClassification.constants import *
from ChestCancerClassification.utils.utils import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_learning_rate=self.params.LEARNING_RATE,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_device=self.params.DEVICE
        )

        return prepare_base_model_config

In [7]:
from torchvision import models
import torch.nn as nn
import torch


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model = models.densenet121(weights=self.config.params_weights)
        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(
        model, classes, freeze_all, freeze_till, learning_rate, device
    ):
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for i in range(freeze_till):
                for param in model.features[i].parameters():
                    model.requires_grad = False

        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, classes)
        full_model = model.to(device)

        optimizer = torch.optim.Adam(full_model.parameters(), lr=learning_rate)
        criterion = torch.nn.CrossEntropyLoss().to(device)

        print(full_model)
        return {
            "model_state_dict": full_model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss_fn": criterion,
        }

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=False,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
            device=self.config.params_device,
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model):
        torch.save(model, path)

In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-03-05 23:23:34,542: INFO: utils: yaml file: config\config.yaml lodded successfully]
[2024-03-05 23:23:34,545: INFO: utils: yaml file: params.yaml lodded successfully]
[2024-03-05 23:23:34,546: INFO: utils: created directory at: artifacts]
[2024-03-05 23:23:34,547: INFO: utils: created directory at: artifacts/prepare_base_model]
DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, m